In [2]:
import route_following_simulation as file
import sys
sys.path.append('/Users/ludvigloite/projects/skole/prosjektoppgave/ship_simulator/')

import matplotlib.pyplot as plt
import numpy as np

from models import ShipModel, ShipConfiguration, EnvironmentConfiguration, \
    MachinerySystemConfiguration, SimulationConfiguration, StaticObstacle, \
    MachineryMode, MachineryModeParams, MachineryModes, ThrottleControllerGains, \
    EngineThrottleFromSpeedSetPoint, HeadingByRouteController, HeadingControllerGains, \
    SpecificFuelConsumptionWartila6L26, SpecificFuelConsumptionBaudouin6M26Dot3, LosParameters, \
    SeachartSimulationConfiguration

import pandas as pd
import shapely.geometry as geo
from random import random
from tqdm import tqdm

failure_modes= ['FM1: Full Blackout', 'FM2: 80% Power Loss', 'FM3: 50% Power Loss', 'FM4: Rudder Freeze']
probabilities = {'FM1: Full Blackout':0.0001, 'FM2: 80% Power Loss':0.0002, 'FM3: 50% Power Loss':0.0003, 'FM4: Rudder Freeze':0.001}
reductionFactor = 500

#TODO: Mistenker at throttle control ikke funker. Vet ikke helt med rudder heller. Problemer med integratoren!
#TODO: Motor shaft speed og rudder har ufysiske raske endringer

simulation_locations = ["hestholmen_east", "hestholmen_west", "tautra_north_1", "tautra_north_2", "tautra_north_3","tautra_south_1", "tautra_south_2", "tautra_south_3"]
simulation_location = simulation_locations[5]
sim_location = "tautra"
#simulation_locations_used = simulation_locations[4:5]
simulation_locations_used = [i for i in simulation_locations if sim_location in i]

nu_of_simulations = 100

draft_of_ship = 6 #Typical draft around 6m: https://horizonship.com/ship/80m-dp2-platform-supply-vessel-1-of-3-sister-ships-2020-dwt-3500/
safety_margin = 50

show_seacharts = False
show_plots = False
save_results_to_csv = False

save_risky_result_svg = False
risky_result_svg_name = ''

simulation_data = {}

simulation_data[simulation_location] = {}
simulation_data[simulation_location]["consequence"] = []




In [3]:
route_txt = "route_seacharts.txt"
init_north_pos = 6955000
init_east_pos = 33100
size_map_east = 18000
size_map_north = 10124
center_map_east = 36580
center_map_north = 6960000
init_yaw_angle = 90 * np.pi / 180

new_data= False
route_txt = 'route_'+simulation_location+'.txt'


if simulation_location.split('_')[0] == "hestholmen":
    init_north_pos = 6993973
    init_east_pos = 75678
    size_map_east = 18000
    size_map_north = 10124
    center_map_east = 74826
    center_map_north = 6989614
    init_yaw_angle = 180 * np.pi / 180
elif simulation_location.split('_')[0] == "tautra":
    init_north_pos = 6974308
    init_east_pos = 81150
    size_map_east = 18000
    size_map_north = 10124
    center_map_east = 85814
    center_map_north = 6974797
    init_yaw_angle = 90 * np.pi / 180



main_engine_capacity = 2160e3
diesel_gen_capacity = 510e3
hybrid_shaft_gen_as_generator = 'GEN'
hybrid_shaft_gen_as_motor = 'MOTOR'
hybrid_shaft_gen_as_offline = 'OFF'

time_step = 0.5

env_config = EnvironmentConfiguration(
    current_velocity_component_from_north=-2,#-2,
    current_velocity_component_from_east=2,#-2,
    wind_speed=15,#15,
    wind_direction=0
)

seacharts_setup = SeachartSimulationConfiguration(
    size_of_map_east = size_map_east,
    size_of_map_north = size_map_north,
    center_of_map_east = center_map_east,
    center_of_map_north = center_map_north,
    database_file_names = ['More_og_Romsdal.gdb'],
    new_data = new_data,
    border = True,
    route_txt = route_txt,
    waypoint_color = 'green',
    draft_of_ship = draft_of_ship, #Typical draft around 6m: https://horizonship.com/ship/80m-dp2-platform-supply-vessel-1-of-3-sister-ships-2020-dwt-3500/
    safety_margin = safety_margin,
    wind_arrow_drawing_coefficient = 20,
    current_arrow_drawing_coefficient = 1000,
    current_wind_arrow_offset = 500,
    wind_arrow_color = 'black',
    current_arrow_color = 'yellow',
    able_to_crash = False
)

In [4]:
ship_config = ShipConfiguration(
    coefficient_of_deadweight_to_displacement=0.7,
    bunkers=200000,
    ballast=200000,
    length_of_ship=80,
    width_of_ship=16,
    added_mass_coefficient_in_surge=0.4,
    added_mass_coefficient_in_sway=0.4,
    added_mass_coefficient_in_yaw=0.4,
    dead_weight_tonnage=3850000,
    mass_over_linear_friction_coefficient_in_surge=130,
    mass_over_linear_friction_coefficient_in_sway=18,
    mass_over_linear_friction_coefficient_in_yaw=90,
    nonlinear_friction_coefficient__in_surge=2400,
    nonlinear_friction_coefficient__in_sway=4000,
    nonlinear_friction_coefficient__in_yaw=400
)
mec_mode_params = MachineryModeParams(
    main_engine_capacity=main_engine_capacity,
    electrical_capacity=diesel_gen_capacity,
    shaft_generator_state=hybrid_shaft_gen_as_offline
)
mec_mode = MachineryMode(params=mec_mode_params)
mso_modes = MachineryModes(
    [mec_mode]
)
fuel_spec_me = SpecificFuelConsumptionWartila6L26()
fuel_spec_dg = SpecificFuelConsumptionBaudouin6M26Dot3()
machinery_config = MachinerySystemConfiguration(
    machinery_modes=mso_modes,
    machinery_operating_mode=0,
    linear_friction_main_engine=68,
    linear_friction_hybrid_shaft_generator=57,
    gear_ratio_between_main_engine_and_propeller=0.6,
    gear_ratio_between_hybrid_shaft_generator_and_propeller=0.6,
    propeller_inertia=6000,
    propeller_diameter=3.1,
    propeller_speed_to_torque_coefficient=7.5,
    propeller_speed_to_thrust_force_coefficient=1.7,
    hotel_load=200000,
    rated_speed_main_engine_rpm=1000,
    rudder_angle_to_sway_force_coefficient=50e3,
    rudder_angle_to_yaw_force_coefficient=500e3,
    max_rudder_angle_degrees=30,
    specific_fuel_consumption_coefficients_me=fuel_spec_me.fuel_consumption_coefficients(),
    specific_fuel_consumption_coefficients_dg=fuel_spec_dg.fuel_consumption_coefficients()
)
simulation_setup = SimulationConfiguration(
    initial_north_position_m= init_north_pos, #6955000,#6958000,
    initial_east_position_m= init_east_pos, #33100,#30000,
    initial_yaw_angle_rad=init_yaw_angle, #90 * np.pi / 180, #90 * np.pi / 180,
    initial_forward_speed_m_per_s=7,
    initial_sideways_speed_m_per_s=0,
    initial_yaw_rate_rad_per_s=0,
    integration_step=time_step,
    simulation_time=1200,#1500,#600,
)

In [5]:
ship_model = ShipModel(ship_config=ship_config,
                    machinery_config=machinery_config,
                    environment_config=env_config,
                    seachart_config=seacharts_setup,
                    simulation_config=simulation_setup,
                    initial_propeller_shaft_speed_rad_per_s=400 * np.pi / 30)

desired_forward_speed_meters_per_second = 8.5
meters_between_each_failure_simulation = 20
time_since_last_ship_drawing = 30
duration_of_failure_seconds = 100
duration_of_failure_simulation_seconds = 100

failure_simulation_time_interval = int(meters_between_each_failure_simulation / (desired_forward_speed_meters_per_second*time_step))


# Set up control systems
throttle_controller_gains = ThrottleControllerGains(
    kp_ship_speed=7, ki_ship_speed=0.13, kp_shaft_speed=0.05, ki_shaft_speed=0.005
)
throttle_controller = EngineThrottleFromSpeedSetPoint(
    gains=throttle_controller_gains,
    max_shaft_speed=ship_model.ship_machinery_model.shaft_speed_max,
    time_step=time_step,
    initial_shaft_speed_integral_error=114
)

heading_controller_gains = HeadingControllerGains(kp=4, kd=90, ki=0.01)
los_guidance_parameters = LosParameters(
    radius_of_acceptance=200,
    lookahead_distance=500,
    integral_gain=0.002,
    integrator_windup_limit=4000
)
auto_pilot = HeadingByRouteController(
    route_name=route_txt,
    heading_controller_gains=heading_controller_gains,
    los_parameters=los_guidance_parameters,
    time_step=time_step,
    max_rudder_angle=machinery_config.max_rudder_angle_degrees * np.pi/180
)

ship_color = 'orange'

integrator_term = []
times = []
failures = []

colors = {'FM1: Full Blackout':'red', 'FM2: 80% Power Loss':'green', 'FM3: 50% Power Loss':'yellow', 'FM4: Rudder Freeze':'black'}

pbar = tqdm(total=1200)

while ship_model.int.time < ship_model.int.sim_time and auto_pilot.next_wpt != auto_pilot.prev_wpt:
    pbar.update(time_step)

    # Measure position and speed
    north_position = ship_model.north
    east_position = ship_model.east
    heading = ship_model.yaw_angle
    speed = ship_model.forward_speed

    

    # Find appropriate rudder angle and engine throttle
    rudder_angle = auto_pilot.rudder_angle_from_route(
        north_position=north_position,
        east_position=east_position,
        heading=heading
    )

    throttle = throttle_controller.throttle(
        speed_set_point=desired_forward_speed_meters_per_second,
        measured_speed=speed,
        measured_shaft_speed=speed
    )
    
    #FM1: Full Blackout':0.0001
    #print("time_interval: "+str(failure_simulation_time_interval))
    #print(ship_model.int.time % failure_simulation_time_interval)
        
    if ship_model.int.time % failure_simulation_time_interval == 0:
        throttle_controller, auto_pilot, failures = ship_model.simulate_failure(failure_type='FM1: Full Blackout', nu_of_timesteps=duration_of_failure_simulation_seconds, failure_time_length=duration_of_failure_seconds, auto_pilot=auto_pilot, throttle_controller=throttle_controller, desired_forward_speed_meters_per_second=desired_forward_speed_meters_per_second, ship_color=colors['FM1: Full Blackout'], rudder_angle=rudder_angle, throttle=throttle, failures=failures, printAllFailures=False)
        
    # Consequence Level Evaluation
    ship_color = 'white'

    position = geo.Point(east_position,north_position)
    dist_land = position.distance(ship_model.landGeometry)
    dist_shore = position.distance(ship_model.shoreGeometry)

    minimum_distance_to_hazard = min(dist_land, dist_shore)

    for i in range(len(ship_model.depths)):
        if ship_model.depths[i] <= ship_model.draft_of_ship:
            dist_seabed = position.distance(ship_model.seabedList[ship_model.depths[i]])
            if dist_seabed < minimum_distance_to_hazard:
                minimum_distance_to_hazard = dist_seabed

    if minimum_distance_to_hazard <= ship_model.safety_margin:
        ship_color = 'orange'
    if minimum_distance_to_hazard == 0 and ship_model.able_to_crash:
        ship_model.add_vessel_drawing(east_position=east_position, north_position=north_position, heading=heading, ship_color=ship_color)
        ship_model.enc.draw_circle((east_position,north_position), 50, 'red', thickness=10, fill=False)
        break

    # Update and integrate differential equations for current time step
    ship_model.store_simulation_data(throttle, rudder_angle)

    ship_model.update_differentials(engine_throttle=throttle, rudder_angle=rudder_angle)
    ship_model.integrate_differentials()

    integrator_term.append(auto_pilot.navigate.e_ct_int)
    times.append(ship_model.int.time)

    # Make a drawing of the ship from above every 20 second
    if time_since_last_ship_drawing > 30 or auto_pilot.next_wpt == auto_pilot.prev_wpt:
        ship_model.add_vessel_drawing(east_position=east_position, north_position=north_position, heading=heading, ship_color=ship_color)
        ship_model.ship_snap_shot()
        time_since_last_ship_drawing = 0
    time_since_last_ship_drawing += ship_model.int.dt
    # Progress time variable to the next time step
    ship_model.int.next_time()
    
pbar.close()
    

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 1053.5/1200 [00:33<00:04, 31.75it/s]


In [9]:
print(ship_model.int.time)
print(len(ship_model.simulation_results['fuel consumption [kg]']))
print(ship_model.simulation_results['fuel consumption [kg]'][-1])

# nontight
# 1139.5
# 3236.4979817530843

# tight
# 1053.5
# 2822.99172426093

# 1058.0
# 40728.348140069495

# 1053.5
# 100.96212817262634

# 1053.5
# 83.12556720114925

1053.5
2107
83.12556720114925


In [10]:
ship_model.enc.add_vessels(*ship_model.ship_positions)
ship_model.draw_wind_and_current_arrows()
ship_model.enc.show_display()




KeyboardInterrupt: 

In [ ]:
#print(ship_model.consequence_dicts)
total_consequence = 0
for element in ship_model.consequence_dicts:
    for key, value in element.items():
        if key=='worst_violation' and value=='did_crash':
            total_consequence += 3
        elif key=='worst_violation' and value=='did_violate_safety_margin':
            total_consequence += 1

In [19]:
total_consequence

0

In [20]:
ship_model.consequence_dicts

[{'failure_mode': 'FM1: Full Blackout',
  'starting_time': 0.0,
  'worst_violation': '',
  'ending_time': 100.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 4.0,
  'worst_violation': '',
  'ending_time': 104.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 8.0,
  'worst_violation': '',
  'ending_time': 108.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 12.0,
  'worst_violation': '',
  'ending_time': 112.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 16.0,
  'worst_violation': '',
  'ending_time': 116.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 20.0,
  'worst_violation': '',
  'ending_time': 120.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 24.0,
  'worst_violation': '',
  'ending_time': 124.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 28.0,
  'worst_violation': '',
  'ending_time': 128.0},
 {'failure_mode': 'FM1: Full Blackout',
  'starting_time': 32.0,
  'worst_violation

In [6]:
count_crash = 0
count_safety_margin = 0
for element in ship_model.consequence_dicts:
    #if element['failure_mode'] == 'FM1: Full Blackout':        
    if element['worst_violation']=='did_crash':
        count_crash += 1
    elif element['worst_violation']=='did_violate_safety_margin':
        count_safety_margin += 1
print("nu of crashes:", count_crash)
print("nu of safety margin: ",count_safety_margin)
#print("total failure simulations: 8480")

nu of crashes: 0
nu of safety margin:  0


In [7]:
print(ship_model.int.time)
print(ship_model.simulation_results['fuel consumption [kg]'][-1])

1053.5
83.12556720114925


In [8]:
print("Voyage "+simulation_location+" took "+str(ship_model.int.time/60)+" minutes. Fuel consumption was "+str(ship_model.simulation_results['fuel consumption [kg]'][-1])+" kg/$. Total consequence was "+str(total_consequence))

NameError: name 'total_consequence' is not defined

In [24]:
import pickle

In [38]:
result_dict = {}
result_dict['route'] = simulation_location
result_dict['duration_of_failure_seconds'] = duration_of_failure_seconds
result_dict['duration_of_failure_simulation_seconds'] = duration_of_failure_simulation_seconds
result_dict['fuel_consumption_kgs'] = ship_model.simulation_results['fuel consumption [kg]'][-1]
result_dict['time_usage_seconds'] = ship_model.int.time
result_dict['consequence_dicts'] = ship_model.consequence_dicts

folder_name='test_13_03_2023'

with open('reports/'+folder_name+'/'+'result_dict_'+str(simulation_location)+'_'+str(duration_of_failure_seconds)+'sec_'+str(duration_of_failure_simulation_seconds)+'sec.pkl', 'wb') as f:
    pickle.dump(result_dict, f)

In [39]:
with open('reports/'+folder_name+'/'+'result_dict_'+str(simulation_location)+'_'+str(duration_of_failure_seconds)+'sec_'+str(duration_of_failure_simulation_seconds)+'sec.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [40]:
loaded_dict

{'route': 'tautra_south_tight',
 'duration_of_failure_seconds': 100,
 'duration_of_failure_simulation_seconds': 100,
 'fuel_consumption_kgs': 83.12556720114925,
 'time_usage_seconds': 1053.5,
 'consequence_dicts': [{'failure_mode': 'FM1: Full Blackout',
   'starting_time': 0.0,
   'worst_violation': '',
   'ending_time': 100.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 4.0,
   'worst_violation': '',
   'ending_time': 104.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 8.0,
   'worst_violation': '',
   'ending_time': 108.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 12.0,
   'worst_violation': '',
   'ending_time': 112.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 16.0,
   'worst_violation': '',
   'ending_time': 116.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 20.0,
   'worst_violation': '',
   'ending_time': 120.0},
  {'failure_mode': 'FM1: Full Blackout',
   'starting_time': 24.0,
   'worst